In [3]:
import zipfile
import os

# Define the path to the uploaded zip file
zip_path = '/content/archive (1).zip'

# Copy the uploaded file from the working directory (if needed)
!cp "/mnt/data/archive (1).zip" "$zip_path"

# Create a directory to extract the contents
extract_dir = '/content/extracted_data'
os.makedirs(extract_dir, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Extraction complete. Files are in:", extract_dir)


cp: cannot stat '/mnt/data/archive (1).zip': No such file or directory
Extraction complete. Files are in: /content/extracted_data


In [4]:
import os

# Explore the directory to find the actual path
for root, dirs, files in os.walk("/content/extracted_data"):
    print(root)


/content/extracted_data
/content/extracted_data/Brain_Stroke_CT-SCAN_image
/content/extracted_data/Brain_Stroke_CT-SCAN_image/Test
/content/extracted_data/Brain_Stroke_CT-SCAN_image/Test/Stroke
/content/extracted_data/Brain_Stroke_CT-SCAN_image/Test/Normal
/content/extracted_data/Brain_Stroke_CT-SCAN_image/Validation
/content/extracted_data/Brain_Stroke_CT-SCAN_image/Validation/Stroke
/content/extracted_data/Brain_Stroke_CT-SCAN_image/Validation/Normal
/content/extracted_data/Brain_Stroke_CT-SCAN_image/Train
/content/extracted_data/Brain_Stroke_CT-SCAN_image/Train/Stroke
/content/extracted_data/Brain_Stroke_CT-SCAN_image/Train/Normal


In [5]:
!pip install flwr[simulation]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.0/540.0 kB 35.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.3
    Uninstalling cryptography-43.0.3:
      Successfully uninstalled cryptography-43.0.3
  Attempting uninstall: typer
    Found existing installation: typer 0.15.3
    Uninstalling typer-0.15.

In [6]:
# ✅ Install required package
!pip install split-folders

import os
import splitfolders

# ✅ Define source and destination paths
original_data_dir = "/content/extracted_data/Brain_Stroke_CT-SCAN_image/Train"  # Your original full dataset
output_dir = "/content/Brain_Stroke_CT-SCAN_image_Split"         # Will contain Hospital_1, Hospital_2, Hospital_3

# ✅ Create split folders (3 hospital splits)
splitfolders.ratio(
    original_data_dir,
    output=output_dir,
    seed=42,
    ratio=(0.33, 0.33, 0.34),
    group_prefix=None,
    move=False  # Set to True if you want to move instead of copy
)

# ✅ Rename output folders to simulate hospitals
os.rename(os.path.join(output_dir, "train"), os.path.join(output_dir, "Hospital_1"))
os.rename(os.path.join(output_dir, "val"), os.path.join(output_dir, "Hospital_2"))
os.rename(os.path.join(output_dir, "test"), os.path.join(output_dir, "Hospital_3"))

print("✅ Dataset successfully split into 3 simulated hospitals!")


Copying files: 1843 files [00:00, 6611.60 files/s]

✅ Dataset successfully split into 3 simulated hospitals!


In [9]:
import os
import shutil
import random
from pathlib import Path

# Base directory
base_dir = 'Brain_Stroke_CT-SCAN_image_Split'

# Define split ratio
train_ratio = 0.8

# Loop through each hospital
for hospital in ['Hospital_1', 'Hospital_2', 'Hospital_3']:
    hospital_path = os.path.join(base_dir, hospital)

    for class_name in ['Normal', 'Stroke']:
        class_path = os.path.join(hospital_path, class_name)
        images = os.listdir(class_path)
        random.shuffle(images)

        split_idx = int(len(images) * train_ratio)
        train_images = images[:split_idx]
        test_images = images[split_idx:]

        # Create destination folders
        for split, split_images in zip(['Train', 'Test'], [train_images, test_images]):
            dest_dir = os.path.join(hospital_path, split, class_name)
            os.makedirs(dest_dir, exist_ok=True)

            # Move images
            for img in split_images:
                src_path = os.path.join(class_path, img)
                dst_path = os.path.join(dest_dir, img)
                shutil.move(src_path, dst_path)

        # Remove the original class folder if empty
        if not os.listdir(class_path):
            os.rmdir(class_path)

print("✅ Dataset reorganized into Train/Test folders per hospital.")


✅ Dataset reorganized into Train/Test folders per hospital.


In [ ]:
pip uninstall tensorflow-gpu
pip install tensorflow


In [12]:
# 📦 Imports
import flwr as fl
import tensorflow as tf
import os
from typing import Tuple
import numpy as np
from sklearn.metrics import accuracy_score

# 📁 Dataset directories (ensure these are uploaded to /content/)
base_dir = "/content/Brain_Stroke_CT-SCAN_image_Split"
clients_dirs = [
    os.path.join(base_dir, "Hospital_1"),
    os.path.join(base_dir, "Hospital_2"),
    os.path.join(base_dir, "Hospital_3"),
]

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

def load_dataset(data_dir: str):
    ds = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        label_mode="binary",
        shuffle=True,
        seed=42
    )
    ds = ds.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))
    return ds

# 🧠 Model definition (CNN or use a pretrained one if needed)
def build_model() -> tf.keras.Model:
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(224, 224, 3)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(64, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy'],
    )
    return model

# 🌍 Define Flower Client
class CTClient(fl.client.NumPyClient):
    def __init__(self, train_ds, test_ds):
        self.model = build_model()
        self.train = train_ds
        self.test = test_ds

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.train, epochs=1, verbose=0)
        return self.model.get_weights(), len(self.train), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, accuracy = self.model.evaluate(self.test, verbose=0)
        return loss, len(self.test), {"accuracy": accuracy}

# 🔁 Client function
def client_fn(cid: str) -> fl.client.Client:
    cid = int(cid)
    client_dir = clients_dirs[cid]
    train_ds = load_dataset(os.path.join(client_dir, "Train"))
    test_ds = load_dataset(os.path.join(client_dir, "Test"))
    return CTClient(train_ds, test_ds)

# 🚀 Start federated simulation
if __name__ == "__main__":
    import ray
    ray.shutdown()
    ray.init(ignore_reinit_error=True)

    fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=3,
        config=fl.server.ServerConfig(num_rounds=3),
        client_resources={"num_cpus": 1}
    )

    ray.shutdown()


2025-05-13 15:02:54,958	INFO worker.py:1771 -- Started a local Ray instance.
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=3, no round_timeout
2025-05-13 15:03:02,769	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'object_store_memory': 3962663731.0, 'node:172.28.0.12': 1.0, 'memory': 7925327463.0, 'node:__internal_head__': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus':

(ClientAppActor pid=13060) Found 485 files belonging to 2 classes.
(ClientAppActor pid=13060) Found 122 files belonging to 2 classes.


(ClientAppActor pid=13060) /usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
(ClientAppActor pid=13060)   super().__init__(activity_regularizer=activity_regularizer, **kwargs)
(ClientAppActor pid=13060) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
INFO :      Received initial parameters from one random client
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 3)
(ClientAppActor pid=13060) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a 

(ClientAppActor pid=13060) Found 485 files belonging to 2 classes.
(ClientAppActor pid=13060) Found 122 files belonging to 2 classes.


(ClientAppActor pid=13059) 
(ClientAppActor pid=13059)         
(ClientAppActor pid=13060) WARNING:tensorflow:AutoGraph could not transform <function load_dataset.<locals>.<lambda> at 0x7a9329ecd800> and will run it as-is.
(ClientAppActor pid=13060) Cause: could not parse the source code of <function load_dataset.<locals>.<lambda> at 0x7a9329ecd800>: no matching AST found among candidates:
(ClientAppActor pid=13060) 
(ClientAppActor pid=13060) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(ClientAppActor pid=13060) WARNING:tensorflow:AutoGraph could not transform <function load_dataset.<locals>.<lambda> at 0x7a9329f4d1c0> and will run it as-is.
(ClientAppActor pid=13060) Cause: could not parse the source code of <function load_dataset.<locals>.<lambda> at 0x7a9329f4d1c0>: no matching AST found among candidates:
(ClientAppActor pid=13060) 
(ClientAppActor pid=13060) To silence this warning, decorate the function with @tf.autograph.experime

...

(ClientAppActor pid=13060) 
(ClientAppActor pid=13060) 
(ClientAppActor pid=13060) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      configure_evaluate: strategy sampled 3 clients (out of 3)
(ClientAppActor pid=13060) 
(ClientAppActor pid=13060)         
(ClientAppActor pid=13060) WARNING:tensorflow:AutoGraph could not transform <function load_dataset.<locals>.<lambda> at 0x7a9319576ca0> and will run it as-is. [repeated 2x across cluster]
(ClientAppActor pid=13060) Cause: could not parse the source code of <function load_dataset.<locals>.<lambda> at 0x7a9319576ca0>: no matching AST found among candidates: [repeated 2x across cluster]
(ClientAppActor pid=13060) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert [

(ClientAppActor pid=13060) Found 485 files belonging to 2 classes. [repeated 2x across cluster]


(ClientAppActor pid=13059) 
(ClientAppActor pid=13059) 
(ClientAppActor pid=13060) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


(ClientAppActor pid=13059) Found 122 files belonging to 2 classes. [repeated 5x across cluster]


(ClientAppActor pid=13059) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=13059) 
(ClientAppActor pid=13059)             This is a deprecated feature. It will be removed
(ClientAppActor pid=13059)             entirely in future versions of Flower.
(ClientAppActor pid=13059)         
(ClientAppActor pid=13059) 
(ClientAppActor pid=13059) 
(ClientAppActor pid=13059) WARNING:tensorflow:AutoGraph could not transform <function load_dataset.<locals>.<lambda> at 0x7d5aabad4cc0> and will run it as-is. [repeated 6x across cluster]
(ClientAppActor pid=13059) Cause: could not parse the source code of <function load_dataset.<locals>.<lambda> at 0x7d5aabad4cc0>: no matching AST found among candidates: [repeated 6x across cluster]
(ClientAppActor pid=13059) To silence this warning,